# 2-Opt Local Search Heuristic
**Author: Dr Bruce Cox  <br/>
Last Revsision: 20 Sept 2021**

This code takes in any arbitrary sized symmetric matrix of distances for the Traveleing Salesman Problem and constructs a valid tour, using Nearest Neighbor. It then performs a 2-opt search and returns revised tour along with cost.  

## Nearest Neighbor

Any construction heuristic could be used.  In fact we could just randomly create a tour. 

In [1]:
import pandas as pd

In [2]:
tsp_data = pd.read_excel("lab data.xlsx", index_col=None)
print(tsp_data.shape)
print(tsp_data)

(15, 15)
    0    1   2   3   4   5    6   7    8   9   10  11  12  13   14
0    0   91  15  99  26  93   69  20   98  62  90  47  23  77    4
1   91    0  50  20  40  59  100  59   13  84  96  68  54  11    1
2   15   50   0  76  96  40    4  53   93  62  59  95  23  97   53
3   99   20  76   0  75  73    5  80   57  12  65  32  94  76   24
4   26   40  96  75   0  65   28  45   12  82  85   3  49  47   80
5   93   59  40  73  65   0    9  81   51  28  93  90  60  73   88
6   69  100   4   5  28   9    0   6   65  12  35   8  61  89   44
7   20   59  53  80  45  81    6   0   77  34   2  36  49  67   99
8   98   13  93  57  12  51   65  77    0  27  62  68  28  38  100
9   62   84  62  12  82  28   12  34   27   0   7  28  49   8    9
10  90   96  59  65  85  93   35   2   62   7   0  97  95  98   26
11  47   68  95  32   3  90    8  36   68  28  97   0  74  34   89
12  23   54  23  94  49  60   61  49   28  49  95  74   0  55   78
13  77   11  97  76  47  73   89  67   38   8  98  34

In [3]:
def valid(tour):
    valid_nbrs = [element for element in range(len(tsp_data)) if element not in tour]
    #print("Valid neighbors : \n" ,valid_nbrs)
    return valid_nbrs

In [4]:
def nearest(current_city):
    smallest = 100**10
    min_position = 0 
    
    for row in tsp_data.itertuples():
        if row[0] in valid(tour):
            if(smallest > row[current_city + 1]):
                smallest = row[current_city + 1]
                min_position = row[0]
    
    return min_position, smallest

In [5]:
def NN(starting_city, tour):   
    tour_length = 0
    current_city = starting_city
    tour.append(current_city)   #add starting city to tour    
    
    for i in range(len(tsp_data)-1):
        min_position, smallest = nearest(current_city)
        tour.append(min_position)
        current_city = min_position
        tour_length += smallest
    
    #The TSP requires that the salesman goes back home at the end.
    tour.append(starting_city)
    tour_length += tsp_data[tour[len(tsp_data)-1]][starting_city]
    
    return tour, tour_length

In [6]:
tour = []
tour_length = 0

NN(1,tour)

([1, 14, 0, 2, 6, 3, 9, 10, 7, 11, 4, 8, 12, 13, 5, 1], 316)

## 2-Opt Local Search Heuristic
The following 2-opt heuristic follows the pseduo code provided in class.  Which also matches that found on Wikipedia, and the following website. </br>

https://en.wikipedia.org/wiki/2-opt </br>
http://pedrohfsd.com/2017/08/09/2opt-part1.html </br>


## Calculate Tour Costs
Given a tour as a list use the distance matrix to return it's cost.

In [7]:
len(tour)

16

In [8]:
tour

[1, 14, 0, 2, 6, 3, 9, 10, 7, 11, 4, 8, 12, 13, 5, 1]

Below I just check my logic to ensure my indices are working as expected.  
**Since Python is 0 index, 14 is actually the last leg (i.e., returning home).**

I'm using the Pandas Dataframe.iat[] method `output = dataframe.iat[row, column]`

In [9]:
i=14
print("The distance from city", tour[i], "to city", tour[i+1], "is", tsp_data.iat[tour[i],tour[i+1]])

The distance from city 5 to city 1 is 59


Note 1: The assumed form of the tour includes a return back to starting node.  This is necessary for a valid TSP tour and is assumed for following cost function.

Note 2: The loop goes to `len(tour)-1` because Python goes up to but not including the last element of the loop. 

In [10]:
def cost(tour):
    cost = 0
    for i in range(0, len(tour)-1):  #Python is 0 index and goes up to *but not including* last element in range
        cost += tsp_data.iat[tour[i],tour[i+1]]
    return cost

In [11]:
cost(tour)

316

### 2 OPT Swap
Given a tour as a list perform all two opt swaps and return best.

Test indices and see what happens with swaps.  What range of initial cities is appropriate?  What range of end cities is appropriate?  This is honestly the hardest part of 2-opt!!

Recall that Python is *"up to but not including"* the last element.  Thus I am choosing to look at the segment of the tour between $i$ and $j-1$ and running that segment in reverse. This construct works with my loop indexing later, I am pretty sure I could change this if I change my later loop too... but I've never really tried since this works.

In [12]:
len(tour)

16

In [13]:
'''
i is my outer loop index
This is the start of the section of tour you are reversing
   i > 0,  else you change starting city  
   i < n-3,  else j=i+2 is starting city again
'''
i=1


'''
j is my inner loop index
This controls the end of the section of tour you are reversing.  
Note that this reverses cities between i and up to *but not including* j
j > i+1, else nothing happens
j < n
'''
j=15  

tour = []
tour_length = 0
tour, tour_length = NN(1,tour)

print("The initial tour was", tour, "with length", tour_length)
print("The i'th city is", tour[i], "...which is included")
print("The j'th city is", tour[j], "...which is excluded")

new_tour = tour[:]
new_tour[i:j] = tour[j-1:i-1:-1]
print("The new tour is", new_tour, "with length", cost(new_tour))

The initial tour was [1, 14, 0, 2, 6, 3, 9, 10, 7, 11, 4, 8, 12, 13, 5, 1] with length 316
The i'th city is 14 ...which is included
The j'th city is 1 ...which is excluded
The new tour is [1, 5, 13, 12, 8, 4, 11, 7, 10, 9, 3, 6, 2, 0, 14, 1] with length 316


Why don't I need to change the starting city within my 2OPT?

Can you convince yourself that the tour 0-1-2-3-0 is the same as the tour 1-2-3-0-1?  

The 2Opt we are implementing fixes the starting city and explores all permutations but due to symmetry of tours that means we are exploring all permuations of reversals. 

In [14]:
def two_opt(tour):
    best_tour = tour[:]
    improved = True
    while improved:
        improved = False
        for i in range(1, len(tour)-3):          #i is index of first city
            for j in range(i+2, len(tour)):      #j is index of second city
                new_tour = tour[:]               #clone tour
                new_tour[i:j] = tour[j-1:i-1:-1] #swap the order of the tour between these two cities
                if cost(new_tour) < cost(best_tour):
                    best_tour = new_tour[:]
                    improved = True
        tour = best_tour[:]
        print("I found a new tour:",best_tour, cost(best_tour))
    return best_tour, cost(best_tour)

### Test 2-Opt

In [23]:
tour = []
tour_length = 0
tour, tour_length = NN(4,tour)

print("The original tour was:", tour)
print("The original tours cost was:", tour_length)

The original tour was: [4, 11, 6, 2, 0, 14, 1, 13, 9, 10, 7, 12, 8, 5, 3, 4]
The original tours cost was: 339


In [24]:
two_opt(tour)

I found a new tour: [4, 11, 6, 7, 10, 9, 13, 1, 14, 0, 2, 12, 8, 5, 3, 4] 315
I found a new tour: [4, 11, 6, 3, 5, 8, 12, 2, 0, 14, 1, 13, 9, 10, 7, 4] 284
I found a new tour: [4, 11, 3, 6, 5, 8, 12, 2, 0, 14, 1, 13, 9, 10, 7, 4] 244
I found a new tour: [4, 11, 3, 6, 5, 7, 10, 9, 13, 1, 14, 0, 2, 12, 8, 4] 241
I found a new tour: [4, 11, 3, 6, 5, 2, 0, 14, 1, 13, 9, 10, 7, 12, 8, 4] 226
I found a new tour: [4, 11, 3, 6, 5, 2, 0, 14, 1, 13, 9, 10, 7, 12, 8, 4] 226


([4, 11, 3, 6, 5, 2, 0, 14, 1, 13, 9, 10, 7, 12, 8, 4], 226)

Play around with the starting tour.  Try feeding 2OPT random starting tours, or ones you make up (that are valid), or using different starting nodes for the NN constructive hueristic?  

You get different ending tours?  Why?  

I get 211 applying 2OPT to the tour output by NN starting at node 1. Can you beat me? 